<a href="https://colab.research.google.com/github/appletreeleaf/Projects/blob/main/Project3_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install wandb

In [ ]:
import torch
import numpy as np
import wandb
import random
from easydict import EasyDict as edict

from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
from tqdm import tqdm
from torch.utils.data import dataloader

## Wandb Setting

In [ ]:
wandb.login() # API key = bcc72b6f4c340bf778bb3310374ab78fc660206e

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
args = edict({'w_project': 'test_project_3_JY',  #goorm_project_3
              'w_entity': 'goorm_team_1',
              'learning_rate': 5e-5,
              'batch_size': {'train': 8,
                             'eval': 8,
                             'test': 8},
              'accumulate': 4,     # 32개씩
              'epochs': 1,
              'seed': 42,
              'model_name': 'skt/kogpt2-base-v2',
              'max_length': 1024})

args['NAME'] = ''f'skt/KoGPT2-base-v2{args.epochs}_max{args.max_length}_lr{args.learning_rate}_{random.randrange(0, 1024)}'
print(args.NAME)

skt/KoGPT2-base-v21_max1024_lr5e-05_278


In [ ]:
BOS = '</s>'
EOS = '</s>'
UNK = '<unk>'


In [ ]:
import torch.nn as nn

class DialogKoGPT2(nn.Module):
  def __init__(self):
    super(DialogKoGPT2, self).__init__()
    self.kogpt2 = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

  def generate(self,
               input_ids,
               do_sample=True,
               max_length= 50,
               top_p=0.92,
               top_k=50,
               temperature= 0.6,
               no_repeat_ngram_size =None,
               num_return_sequences=3,
               early_stopping=False,
               ):
    return self.kogpt2.generate(input_ids,
               do_sample=do_sample,
               max_length=max_length,
               top_p = top_p,
               top_k=top_k,
               temperature=temperature,
               no_repeat_ngram_size= no_repeat_ngram_size,
               num_return_sequences=num_return_sequences,
               early_stopping = early_stopping,
              )

  def forward(self, input, labels = None):
    if labels is not None:
      outputs = self.kogpt2(input, labels=labels)
    else:
      outputs = self.kogpt2(input)

    return outputs

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
class WellnessAutoRegressiveDataset():      # raw data를 모델 입력형식에 맞게끔 변환
  """Wellness Auto Regressive Dataset"""

  def __init__(self,
               file_path1 = "/content/drive/MyDrive/Groom_8th_project3/Data/wellness_dialog_for_autoregressive.txt",
               file_path2 = '/content/drive/MyDrive/Groom_8th_project3/Data/ChatbotData.txt',
               file_path3 = '/content/drive/MyDrive/Groom_8th_project3/Data/data.txt',
               n_ctx = 512, token = tokenizer):
    self.file_path1 = file_path1
    self.file_path2 = file_path2
    self.file_path3 = file_path3
    self.data =[]
    self.q = []
    self.a = []
    self.tokenizer = token


    bos_token_id = [self.tokenizer.bos_token_id] # word to inteager(token)
    eos_token_id = [self.tokenizer.eos_token_id]
    pad_token_id = [self.tokenizer.pad_token_id]

    file1 = open(self.file_path1, 'r', encoding = "utf-8")
    cnt = 0
    while True:
      line = file1.readline() # 한줄씩 읽음
      if not line:            # 다 읽었으면 멈춤
        break
      if cnt > 0:
        datas = line.split(".,") # .,기준으로 Q / A split
        index_of_words = self.tokenizer.encode(datas[0]) + bos_token_id + self.tokenizer.encode(datas[1][:-1])+ eos_token_id
        self.q.append(self.tokenizer.encode(datas[0]))
        self.a.append(self.tokenizer.encode(datas[1][:-1]) + eos_token_id)


        pad_token_len = n_ctx - len(index_of_words)       # padding
        index_of_words += pad_token_id * pad_token_len    # max_len보다 작은 sequence에 padding
        self.data.append(index_of_words)                  # 가공이 끝난 sequence를 data
      cnt += 1
    file1.close()

    cnt1 = 0
    file2 = open(self.file_path2, 'r', encoding='utf-8')
    while True:
      line = file2.readline()
      if not line:
        break
      if cnt > 1:
        datas = line.split(",")
        index_of_words = self.tokenizer.encode(datas[0]) + bos_token_id + self.tokenizer.encode(datas[1][:-1])+ eos_token_id # q + <bos> + a + <eos>
        self.q.append(self.tokenizer.encode(datas[0]))
        self.a.append(self.tokenizer.encode(datas[1][:-1]) + eos_token_id)
        pad_token_len = n_ctx - len(index_of_words)
        index_of_words += pad_token_id * pad_token_len
        self.data.append(index_of_words)
      cnt += 1
    file2.close()

    file3 = open(self.file_path3, 'r', encoding='utf-8')
    while True:
      line = file3.readline()
      if not line:
        break
      if cnt > 2:
        datas = line.split(",")
        index_of_words = self.tokenizer.encode(datas[0]) + bos_token_id + self.tokenizer.encode(datas[1][:-1])+ eos_token_id # q + <bos> + a + <eos>
        self.q.append(self.tokenizer.encode(datas[0]))
        self.a.append(self.tokenizer.encode(datas[1][:-1]) + eos_token_id)
        pad_token_len = n_ctx - len(index_of_words)
        index_of_words += pad_token_id * pad_token_len
        self.data.append(index_of_words)
      cnt += 1
    file3.close()

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    # item = {}
    item = self.data[index]
    # label = self.a[index]
    # item["label"] = label

    return item

In [ ]:
n_epoch = 1        # Num of Epoch
batch_size = 8      # 배치 사이즈
save_step = 100 # 학습 저장 주기
learning_rate = 5e-5  # Learning Rate

checkpoint_path ="/content/drive/MyDrive/Groom_8th_project3/model"
save_ckpt_path = f"{checkpoint_path}/kogpt2-wellnesee-auto-regressive_1.pth"

In [ ]:
wandb.init(project = args.w_project, entity = args.w_entity) # (project name, project_department)

wandb: Currently logged in as: 02younge (goorm_team_1). Use `wandb login --relogin` to force relogin


In [ ]:
wandb.run.name = args.NAME
wandb.config.learning_rate = args.learning_rate
wandb.config.epochs = args.epochs

In [ ]:
dataset= WellnessAutoRegressiveDataset()
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print(dataset.__getitem__(1))
len(dataset)

[9037, 29524, 9510, 12107, 9031, 27511, 25856, 15812, 7489, 10704, 12258, 32738, 8006, 32286, 16959, 12537, 8146, 9183, 7251, 25856, 1, 9265, 7235, 9022, 9032, 7671, 10014, 8711, 25856, 9564, 9723, 11392, 8234, 406, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

130956

In [ ]:
model = DialogKoGPT2()
model.to(device)

DialogKoGPT2(
  (kogpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(51200, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=51200, bias=False)
  )
)

In [ ]:
loss_cross = torch.nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
losses = []
for epoch in range(args.epochs):
    count = 0
    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, data in enumerate(train_loader): # data = 길이 1024의 embedding vector
            optimizer.zero_grad()     # initialize gradient
            data = torch.stack(data)  # Tensor들로 구성된 list이기 때문에 list를 stack을 쌓아주고 (S_L, B)
            data = data.transpose(1, 0) # (1, B, S_L)
            data= data.to(device)

            outputs = model(data, labels=data)
            _, logits = outputs[:2]
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = data[..., 1:].contiguous()

            loss = loss_cross(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()

            losses.append(loss.item())
            wandb.log({'train_loss': np.mean(losses)})


            if count % save_step ==0 or (len(data) < batch_size):
                torch.save({
                    'epoch': epoch,
                    'train_no': count,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss
                }, save_ckpt_path)
                model.kogpt2.save_pretrained(f'/content/drive/MyDrive/Groom_8th_project3{epoch}')
            count += 1
            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")

Train(1): 100%|██████████| 16370/16370 [1:36:47<00:00,  2.82it/s, Loss: 1.928 (2.444)]


##테스트

In [ ]:
!pip install streamlit
!pip install gtts

In [ ]:
import streamlit as st
from typing import Dict, List
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import gtts


tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained("jack-oh/KoGPT2_finetuned_wellness")
model.eval()
st.set_page_config(page_title='Project3', page_icon='translator-icon.png', layout='wide', initial_sidebar_state='expanded')
st.title("Lyric Translator:balloon:")



# text = st.text_area("Enter text:",height=None,max_chars=None,key=None,help="Enter your text here")
text = st.text_area("Enter text:", placeholder="Enter your text here")

option1 = st.selectbox('Input language',
                        ('english' , 'korean'))
option2 = st.selectbox('Output language',
                        ('korean' , 'english'))
Languages = {'english':'en', 'korean':'ko'}
value1 = Languages[option1] # 'en' or 'ko'
value2 = Languages[option2] # 'en' or 'ko'

if st.button('Translate Sentence'):
    if text == "":
        st.warning('Please **enter text** for translation')

    else:
        embeddings = tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt') #GPT의 input
        output = en2kor_model.generate(**embeddings, max_length=50)[0, 1:-1]
        output=en2kor_trg_tokenizer.decode(output, skip_special_tokens=True)


        st.text_area("Translation", value=output, disabled=True)

        converted_audio = gtts.gTTS(output, lang=value2)
        converted_audio.save("translated.mp3")
        audio_file = open('translated.mp3','rb')
        audio_bytes = audio_file.read()
        st.audio(audio_bytes, format='audio')
        st.write("To **download the audio file**, click the kebab menu on the audio bar.")
        st.success("Translation is **successfully** completed!")
        st.balloons()
else:
    pass

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Groom_8th_project3/model/kogpt2-wellnesee-auto-regressive_1.pth")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


OSError: ignored

In [ ]:
U_TKN = '<usr>'
S_TKN = '<sys>'
EOS = '</s>'
MASK = '<unused0>'
PAD = '<pad>'
SENT = '<unused1>'

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
def chat():
    with torch.no_grad():
      print("채팅을 시작합니다. \n")
      qs=[]
      while 1:
          q = input('나 > ').strip()
          qs.append(q) # history 저장

          if q == 'quit':
              break
          a=''
          user = U_TKN + q + SENT + a         # question만 넣음
          encoded = tokenizer.encode(user)
          input_ids = torch.LongTensor(encoded).unsqueeze(dim=0)
          output = model.generate(input_ids,
                                  max_length=50,
                                  num_beams=10,
                                  do_sample=False,
                                  top_k=50,
                                  no_repeat_ngram_size=2,
                                  temperature=0.85)
          a=tokenizer.decode(output[0])
          idx = torch.where(output[0]==tokenizer.encode('<sys>')[0])
          chatbot = tokenizer.decode(output[0][int(idx[0])+1:], skip_special_tokens=True)

          if '답변' in a: # 응, 아니 등이 input으로 들어왔을 때
              a_new = ''
              user = U_TKN + ''.join(qs[-2:]) + SENT + a_new # 직전 history 가지고 와서 sentiment 고려해주기
              encoded = tokenizer.encode(user)
              input_ids = torch.LongTensor(encoded).unsqueeze(dim=0)
              output = model.generate(input_ids,max_length=50,
                                        num_beams=10, do_sample=False,
                                        top_k=50, no_repeat_ngram_size=2,
                                      temperature=0.85)
              a_new = tokenizer.decode(output[0], skip_special_tokens=True)
              idx = torch.where(output[0]==tokenizer.encode('<sys>')[0])
              chatbot = tokenizer.decode(output[0][int(idx[0])+1:], skip_special_tokens=True)

              print("챗봇 > {}".format(chatbot.strip()))

          else:
              print("챗봇 > {}".format(chatbot.strip()))

In [ ]:
chat()

채팅을 시작합니다. 

나 > 친구랑 싸웠어...
챗봇 > 웃음이 줄어들고 눈물도 더 이상 나지 않아 감각이 없이 멍한 듯한 느낌이 드나요?
나 > 응 조금 그런 거 같아
챗봇 > 싸우면서 정 들 거예요.
나 > 무슨 소리야
챗봇 > 직설적으로 통보하는 말보다는 조금 더 부드럽게 권유하고 물어보는 말을 해보세요.
나 > 내가 그랬어 근데 걔는 그러지 않았어
챗봇 > 지속적으로 당신이 못났다고 생각하시나요?
나 > 아니 난 잘났다고 생각해
챗봇 > 지속적으로 당신을 비판하고 자책하시 성공하였다.
당하였다.
 감각이 없이 멍한 듯한 느껴지시나요?
나 > quit
